<a href="https://colab.research.google.com/github/JonathanJuradoS/EspacioEstados/blob/main/8Puzzle_HillClimbing_AStar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔍 8-Puzzle con Hill Climbing y A*

**Objetivo del caso:** Resolver el clásico rompecabezas de 8 fichas, encontrando una secuencia de movimientos que lleve desde un estado inicial desordenado hasta el estado meta ordenado.

**Definición del problema:**
- Tablero 3x3
- Fichas del 1 al 8 y un espacio vacío (0)
- Movimientos: arriba, abajo, izquierda, derecha


In [3]:
# 🔧 Definición del problema y visualización inicial
# Este bloque define el estado inicial y final del rompecabezas de 8. Se usa una lista de 9 elementos (de 0 a 8), donde 0 representa el espacio vacío.
# La función `imprimir_estado` permite visualizar el estado de forma matricial (3x3).

import numpy as np

def es_estado_valido(estado):
    return sorted(estado) == list(range(9))

def imprimir_estado(estado):
    for i in range(0, 9, 3):
        print(estado[i:i+3])
    print()

estado_inicial = [1, 2, 3,
                  4, 0, 6,
                  7, 5, 8]

estado_objetivo = [1, 2, 3,
                   4, 5, 6,
                   7, 8, 0]

imprimir_estado(estado_inicial)
imprimir_estado(estado_objetivo)

[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



## 🚀 Hill Climbing

In [4]:
# 🚀 Hill Climbing
# Este algoritmo busca mejorar el estado actual evaluando sus vecinos. Si encuentra uno mejor (menor heurística), se mueve hacia él. Puede quedar atrapado en óptimos locales si ningún vecino es mejor.
# Heurística usada: Número de fichas fuera de lugar (Hamming)

from copy import deepcopy

def heuristica_hamming(estado, objetivo):
    return sum([1 for i in range(9) if estado[i] != 0 and estado[i] != objetivo[i]])

def mover(estado, direccion):
    nuevo = estado[:]
    idx = nuevo.index(0)
    i, j = divmod(idx, 3)
    if direccion == 'arriba' and i > 0:
        nuevo[idx], nuevo[idx - 3] = nuevo[idx - 3], nuevo[idx]
    elif direccion == 'abajo' and i < 2:
        nuevo[idx], nuevo[idx + 3] = nuevo[idx + 3], nuevo[idx]
    elif direccion == 'izquierda' and j > 0:
        nuevo[idx], nuevo[idx - 1] = nuevo[idx - 1], nuevo[idx]
    elif direccion == 'derecha' and j < 2:
        nuevo[idx], nuevo[idx + 1] = nuevo[idx + 1], nuevo[idx]
    return nuevo

def hill_climbing(estado_inicial, objetivo):
    actual = estado_inicial[:]
    camino = [actual]
    while True:
        vecinos = [mover(actual, d) for d in ['arriba','abajo','izquierda','derecha']]
        vecinos = [v for v in vecinos if v != actual]
        vecino_mejor = min(vecinos, key=lambda x: heuristica_hamming(x, objetivo), default=actual)
        if heuristica_hamming(vecino_mejor, objetivo) >= heuristica_hamming(actual, objetivo):
            break
        actual = vecino_mejor
        camino.append(actual)
    return camino

camino_hc = hill_climbing(estado_inicial, estado_objetivo)
for paso in camino_hc:
    imprimir_estado(paso)
print("Movimientos:", len(camino_hc)-1)

[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Movimientos: 2


## 🌟 A* (A estrella)

In [5]:
# 🌟 A* (A estrella)
# Este algoritmo utiliza el costo acumulado `g(n)` y una heurística `h(n)` para priorizar los estados más prometedores.
# Se garantiza encontrar la solución óptima si la heurística es admisible (como la distancia Manhattan).

import heapq

def heuristica_manhattan(estado, objetivo):
    distancia = 0
    for n in range(1, 9):
        i, j = divmod(estado.index(n), 3)
        x, y = divmod(objetivo.index(n), 3)
        distancia += abs(i - x) + abs(j - y)
    return distancia

def a_estrella(estado_inicial, objetivo):
    visitados = set()
    heap = []
    heapq.heappush(heap, (0 + heuristica_manhattan(estado_inicial, objetivo), 0, estado_inicial, []))

    while heap:
        f, g, actual, camino = heapq.heappop(heap)
        tupla_estado = tuple(actual)
        if tupla_estado in visitados:
            continue
        visitados.add(tupla_estado)
        if actual == objetivo:
            return camino + [actual]
        for d in ['arriba','abajo','izquierda','derecha']:
            vecino = mover(actual, d)
            if tuple(vecino) not in visitados:
                heapq.heappush(heap, (g + 1 + heuristica_manhattan(vecino, objetivo), g + 1, vecino, camino + [actual]))
    return []

camino_astar = a_estrella(estado_inicial, estado_objetivo)
for paso in camino_astar:
    imprimir_estado(paso)
print("Movimientos:", len(camino_astar)-1)

[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Movimientos: 2


## ✅ Conclusiones
- Hill Climbing puede encontrar soluciones rápidas, pero puede quedarse atrapado en óptimos locales.
- A* es más robusto y siempre encuentra la mejor solución si se usa una buena heurística.
- El rompecabezas de 8 es una excelente herramienta para entender la diferencia entre algoritmos de búsqueda local y heurística informada.
- Comparar ambos algoritmos permite reflexionar sobre eficiencia, optimalidad y limitaciones de cada método.